# The Solid Design Principle

## Single Responsibility Principle , Separation of concerns 

If you have a class it should have a primary responsibility and it should not take up other responsibility

In [1]:
class Journal:
    def __init__(self):
        self.entries = []
        self.count=0
    
    def add_entry(self,text):
        self.count +=1
        self.entries.append(f'{self.count}:{text}')
        
    def remove_entry(self,pos):
        del self.entries[pos]
        
    def __str__(self):
        return '\n'.join(self.entries)
    
j = Journal()
j.add_entry('Today is monday')
j.add_entry('I have to go to sleep')

print(f'Journal entries:\n {j}')

Journal entries:
 1:Today is monday
2:I have to go to sleep


- Lets break the Single Responsibility Principle, and give additional work to the Journal class , by asking it to save the Journal to a file.
- Add secondary responsiblity , now we are asking it to persist.
- Any change in the persistance will have to change in all the classes 

In [2]:
class Journal:
    def __init__(self):
        self.entries = []
        self.count=0
    
    def add_entry(self,text):
        self.count +=1
        self.entries.append(f'{self.count}:{text}')
        
    def remove_entry(self,pos):
        del self.entries[pos]
        
    def __str__(self):
        return '\n'.join(self.entries)
    
    ## Add secondary responsiblity , now we are asking it to persist.
    ## Any change in the pers
    def save(self,filename):
        file = open(filename,'w')
        file.write(srt(self))
        file.close()
    
    def load(self,filename):
        pass
    
j = Journal()
j.add_entry('Today is monday')
j.add_entry('I have to go to sleep')

print(f'Journal entries:\n {j}')

Journal entries:
 1:Today is monday
2:I have to go to sleep


To fix this issue we break the code and put it into another class called Persistance manager
This is to not create a anti-pattern, and avoid creating a God object . Not to put everything into one class.
A class should always have a single reason to change and that should be related to it primary responsibility.

In [3]:
class Journal:
    def __init__(self):
        self.entries = []
        self.count=0
    
    def add_entry(self,text):
        self.count +=1
        self.entries.append(f'{self.count}:{text}')
        
    def remove_entry(self,pos):
        del self.entries[pos]
        
    def __str__(self):
        return '\n'.join(self.entries)
    
    ## Add secondary responsiblity , now we are asking it to persist.
    ## Any change in the persitance will have to chage all the classes 
#     def save(self,filename):
#         file = open(filename,'w')
#         file.write(srt(self))
#         file.close()
    
#     def load(self,filename):
#         pass
    
class PersistanceManager:
    @staticmethod
    def save_to_file(journal,filename):
        file = open(filename,'w')
        file.write(str(journal))
        file.close()
    
j = Journal()
j.add_entry('Today is monday')
j.add_entry('I have to go to sleep')

print(f'Journal entries:\n {j}')

file = r'.\Journal.txt'
PersistanceManager.save_to_file(j,file)

Journal entries:
 1:Today is monday
2:I have to go to sleep


In [4]:
with open(file) as fh:
    print(fh.read())

1:Today is monday
2:I have to go to sleep


## Open Closed Principle 

## Interface Segregation Principle

In [6]:
## Dependency Inversion Principle